## Introduction
Semantic textual similarity deals with determining how similar a pair of text documents are. The goal of the first task is to implement a new architecture by combining the ideas from papers
- Siamese Recurrent Architectures for Learning Sentence Similarity, Jonas Mueller et. al (will be referred as the AAAI paper)
- A Structured Self-Attentive Sentence Embedding, Zhouhan Lin et. al (will be referred as the ICLR paper) <br/><br/>
Furthermore, you'd be evaluating whether the new architecture improves the results of **Siamese Recurrent Architectures for Learning Sentence Similarity, Jonas Mueller et. al**. Your overall network architecture should look similar to the following figure. 
![Untitled%20Diagram.drawio%20%281%29.png](https://raw.githubusercontent.com/shahrukhx01/ocr-test/main/download.png)
<br/><br/>


Moreover, you'd be required to implement further helper functions that these papers propose i.e., attention penalty term for loss, etc.

### SICK dataset
We will use SICK dataset throughout the project (at least in the first two tasks). To get more information about the dataset you can refer to the original [paper](http://www.lrec-conf.org/proceedings/lrec2014/pdf/363_Paper.pdf) on the dataset. You can download the dataset using one of the following links:
- [dataset page 1](https://marcobaroni.org/composes/sick.html)
- [dataset page 2](https://huggingface.co/datasets/sick)    

The relevant columns for the project are `sentence_A`, `sentence_B`, `relatedness_score`, where `relatedness_score` is the label. <br><br>
**Hint: For each task make sure to decide whether the label should be normalized or not.**<br><br>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import warnings
import sts_data
from importlib import reload
from test import evaluate_test_set
warnings.filterwarnings('ignore')

## Part 1. Data pipeline (3 points)
Before starting working on the model, we must configure the data pipeline to load the data in the correct format. Please, implement the functions for processing the data.

### Part 1.1 Loading and preprocessing the data (1 point)
Download the SICK dataset and store it in [pandas](https://pandas.pydata.org/docs/index.html) `Dataframe`'s. You should use the official data split.  

Implement `load_data` method of `STSData` class in `sts_data.py`. The method must download the dataset and perform basic preprocessing. Minimal preprocessing required:  
1. normalize text to lower case
2. remove punctuations  
3. remove [stopwords](https://en.wikipedia.org/wiki/Stop_word) - we provided you with the list of English stopwords.
4. Optionally, any other preprocessing that you deem necessary.

All the preprocessing code must be contained in the `preprocessing.py` file.  
You can use Hugginface's [datasets library](https://huggingface.co/docs/datasets/) for easy dataset download.

### Part 1.2 Building vocabulary (1 point)
Before we can feed our text to the model it must be vectorized. We use 300 dimensional pretrained [FastText embeddings](https://fasttext.cc/docs/en/english-vectors.html) for mapping words to vectors. To know more general information about embeddings you can refer to [this video](https://www.youtube.com/watch?v=ERibwqs9p38) (even though, we use different types of embeddings - FastText vs Word2Vec described in the video - the general purpose of them is the same).  
In order to apply the embedding, we must first construct the vocabulary for data. Complete the `create_vocab` method of `STSData` class in `sts_data.py` where you concatenate each sentence pair, tokenize it and construct the vocabulary for the whole training data. You should use [torchtext](https://torchtext.readthedocs.io/en/latest/data.html
) for processing the data. For tokenization, you can use any library (or write your own tokenizer), but we recommend you to use tokenizer by [spacy](https://spacy.io/). Use the `fasttext.simple.300d` as pretrained vectors.  
In the end, you must have a vocabulary object capable of mapping your input to corresponding vectors. Remember that the vocabulary is created using only training data (not touching validation or test data).

### Part 1.3 Creating DataLoader (1 point)
Implement `get_data_loader` method of `STSData` class in `sts_data.py`. It must perform the following operations on each of the data splits:
1. vectorize each pair of the sentences by replacing all tokens with their index in vocabulary
2. normalize labels
3. convert everything to PyTorch tensors
4. pad every sentence so that all of them have the same length
5. create `STSDataset` from `dataset.py`
6. create PyTorch DataLoader out of the created dataset. 


We have provided you with the interfaces of possible helper functions, but you can change them as you need.   
In the end, you must have 3 data loaders for each of the splits.

In [3]:
reload(sts_data)
from sts_data import STSData
import warnings
warnings.filterwarnings('ignore')
columns_mapping = {
        "sent1": "sentence_A",
        "sent2": "sentence_B",
        "label": "relatedness_score",
    }
dataset_name = "sick"
sick_data = STSData(
    dataset_name=dataset_name,
    columns_mapping=columns_mapping,
    normalize_labels=True,
    normalization_const=5.0,
)
batch_size = 64
sick_dataloaders = sick_data.get_data_loader(batch_size=batch_size)

INFO:root:loading and preprocessing data...


Dataset sick downloaded and prepared to /home/ibrahimssd/.cache/huggingface/datasets/sick/default/0.0.0/c6b3b0b44eb84b134851396d6d464e5cb8f026960519d640e087fe33472626db. Subsequent calls will reuse this data.


INFO:root:reading and preprocessing data completed...
INFO:root:creating vocabulary...
INFO:torchtext.vocab:Loading vectors from .vector_cache/wiki.simple.vec.pt
INFO:root:creating vocabulary completed...


## Part 2. Model Configuration & Hyperparameter Tuning (3 points)
In this part, you are required to define a model capable of learning self-attentive sentence embeddings described in [this ICLR paper](https://arxiv.org/pdf/1703.03130.pdf). The sentence embedding learned by this model will be used for computing the similarity score instead of the simpler embeddings described in the original AAAI paper.  
Please familiarize yourself with the model described in the ICLR paper and implement `SiameseBiLSTMAttention` and `SelfAttention` classes in `siamese_lstm_attention.py`. Remember that you must run the model on each sentence in the sentence pair to calculate the similarity between them. You can use `similarity_score` from `utils.py` to compute the similarity score between two sentences. 
  
To get more theoretical information about attention mechanisms you can refer to [this chapter](https://web.stanford.edu/~jurafsky/slp3/10.pdf) of ["Speech and Language Processing" book](https://web.stanford.edu/~jurafsky/slp3/) by Dan Jurafsky and James H. Martin, where the attention mechanism is described in the context of the machine translation task. 

Finally, once your implementation works on the default parameters stated below, make sure to perform **hyperparameter tuning** to find the best combination of hyperparameters.

In [4]:
output_size = 1
hidden_size = 64
vocab_size = len(sick_data.vocab)
embedding_size = 300
embedding_weights = sick_data.vocab.vectors
lstm_layers = 4
learning_rate = 0.01
fc_hidden_size = 64
max_epochs = 100
bidirectional = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## self attention config
self_attention_config = {
    "hidden_size": 150,  ## refers to variable 'da' in the ICLR paper
    "output_size": 20,  ## refers to variable 'r' in the ICLR paper
    "penalty": 2e-1,  ## refers to penalty coefficient term in the ICLR paper
}

In [5]:
## init siamese lstm

from siamese_lstm_attention import SiameseBiLSTMAttention

siamese_lstm_attention = SiameseBiLSTMAttention(
    batch_size=batch_size,
    output_size=output_size,
    hidden_size=hidden_size,
    vocab_size=vocab_size,
    embedding_size=embedding_size,
    embedding_weights=embedding_weights,
    lstm_layers=lstm_layers,
    self_attention_config=self_attention_config,
    fc_hidden_size=fc_hidden_size,
    device=device,
    bidirectional=bidirectional,
)

# optimizer = torch.optim.SGD(siamese_lstm_attention.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(siamese_lstm_attention.parameters(),lr=learning_rate, betas=(0.9, 0.999), eps=1e-08)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
## move model to device
siamese_lstm_attention.to(device)

SiameseBiLSTMAttention(
  (embeddings): Embedding(2029, 300)
  (lookup_table): Embedding(2029, 300)
  (bi_lstm): LSTM(300, 64, num_layers=4, bias=False, batch_first=True, bidirectional=True)
  (SelfAtt): SelfAttention(
    (ws1): Linear(in_features=128, out_features=150, bias=False)
    (ws2): Linear(in_features=150, out_features=20, bias=False)
    (tanh): Tanh()
    (softmax): Softmax(dim=None)
  )
  (normalize): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=2560, out_features=64, bias=True)
  (tanh): Tanh()
)

## Part 3. Training (2 points)  
Perform the final training of the model by implementing functions in `train.py` after setting values of your best-chosen hyperparameters. Note you can use the same training function when performing hyperparameter tuning.
- **What is a good choice of performance metric here for evaluating your model?** [Max 2-3 lines]

- The main metric used here is the Pearson correlation to evaluate the linear relationship between the two continuous variables (predictions and scores). So pearson coefficient correlation is a good choice for this task based on the state-of-art papers.

- **What other performance evaluation metric can we use here for this task? Motivate your answer.**[Max 2-3 lines]

- Spearman correlation could also be used as a metric. It is based on the ranked values and evaluates the monotonic relationship as well. Explained_variance score could also be a choice to explain the variance between correlated continuous values. All of the mentioned metrics are a good tool for evaluating the relation between two continous group.


In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from train import train_model
import warnings
import pickle

warnings.filterwarnings('ignore')
siamese_lstm_attention = train_model(
    model=siamese_lstm_attention,
    optimizer=optimizer,
    scheduler=scheduler,
    dataloader=sick_dataloaders,
    data=sick_data,
    max_epochs=max_epochs,
    clip=True,
    config_dict={
        "device": device,
        "model_name": "siamese_lstm_attention",
        "self_attention_config": self_attention_config,
    },
)


#load and save model
model_filename = "lstm_model.sav"
saved_model = pickle.dump(siamese_lstm_attention, open(model_filename,'wb'))
print('Model is saved into to disk successfully')

  0%|          | 0/50 [00:00<?, ?it/s]INFO:root:Evaluating accuracy on dev set
INFO:root:new model saved
INFO:root:Train loss: 6.4187774658203125 - Train pearson score: 0.13878308269081924 -- Validation loss: 5.561148643493652 - Validation pearson score: 0.11024243164888813- Validation                  p_value: 0.00012070761559507383
  2%|▏         | 1/50 [00:42<34:39, 42.45s/it]

[0/49] train_loss: 6.419, train_score: 0.139 


INFO:root:Evaluating accuracy on dev set
INFO:root:new model saved
INFO:root:Train loss: 5.099574089050293 - Train pearson score: 0.10736160276461369 -- Validation loss: 4.620316982269287 - Validation pearson score: 0.2096014655316269- Validation                  p_value: 4.16795600158878e-06
  4%|▍         | 2/50 [01:23<33:30, 41.88s/it]

[1/49] train_loss: 5.100, train_score: 0.107 


INFO:root:Evaluating accuracy on dev set
INFO:root:Train loss: 4.503004550933838 - Train pearson score: 0.10050299520823366 -- Validation loss: 4.283022880554199 - Validation pearson score: 0.1159030148036601- Validation                  p_value: 0.0033209428244082326
  6%|▌         | 3/50 [02:01<32:06, 40.99s/it]

...

[45/49] train_loss: 3.338, train_score: 0.350 


INFO:root:Evaluating accuracy on dev set
INFO:root:new model saved
INFO:root:Train loss: 3.34891939163208 - Train pearson score: 0.3325580007540644 -- Validation loss: 3.3572938442230225 - Validation pearson score: 0.2642601410719377- Validation                  p_value: 7.58171678819945e-09
 94%|█████████▍| 47/50 [25:05<01:44, 34.80s/it]

[46/49] train_loss: 3.349, train_score: 0.333 


INFO:root:Evaluating accuracy on dev set
INFO:root:Train loss: 3.3391799926757812 - Train pearson score: 0.336652247386211 -- Validation loss: 3.3584818840026855 - Validation pearson score: 0.2520584108704959- Validation                  p_value: 1.5717649448633558e-07
 96%|█████████▌| 48/50 [25:41<01:09, 34.92s/it]

[47/49] train_loss: 3.339, train_score: 0.337 


[99/99] train_loss: 3.220, train_score: 0.877 
Final score: 0.877, expected 1.000

Train loss: 3.220442295074463 - Train pearson score: 0.8765596209715861 -- Validation loss: 3.2523105144500732 - Validation pearson score: 0.6362855344577428- Validation                  p_value: 2.162431682382807e-35

## Part 4. Evaluation and Analysis (2 points)  
Implement function evaluate_test_set to calculate the final accuracy of the performance evaluation metric on the test data.  
Compare the result with the original AAAI paper. Сomment on effect of penalty loss on model capacity. Did the inclusion of the self-attention block improve the results? If yes, then how? Can you think of additional techniques to improve the results? Briefly answer these questions in the markdown cells.

- The results obtained with this model are worse compared to the original paper , which means adding self-attention layer doesn't improve the model. The AAAI paper outperforms the baseline model.
- To improve the base results several approches could be implemented: we can remove self-attention layer and add a transformer instead or use both at the same time.
- We can apply transfer learning to train the model to classifiy sentences using labels column in sick dataset , and then fine-tune the output model on our main target task.
- The affect of penalty loss is to penalize specific weights in order to diffrentiate between different weight vectors accross different hops of attention(reduce redundancy in the embedding matrix and add diversity to weight vectors) , which reduces model flexibility.
- In other words penalty term reduces the model capacity by reducing its variance and introduce some bias in the model (because it penalizes some parameters) to prevent overfitting. 

In [ ]:
import pickle
import warnings
model_filename = "lstm_model.sav"
from test import evaluate_test_set
lstm_model = pickle.load(open(model_filename, 'rb'))
warnings.filterwarnings('ignore')

evaluate_test_set(
    model=lstm_model,
    data_loader=sick_dataloaders,
    config_dict={
        "device": device,
        "model_name": "siamese_lstm_attention",
        "self_attention_config": self_attention_config,
    },
)

train_loss: 3.220, train_score: 0.877 Final score: 0.877, expected 1.000

test_loss: 0.257, spearman_score: 0.328 , pearson_score: 0.336